Расчет силы действующей на проводник со стороны переменного магнитного поля

А.Ю.Дроздов

In [7]:
from IPython.display import display, Math, Latex

In [8]:
mu = var('mu')
assume(mu, 'real')
assume(mu>0)

epsilon = var('epsilon')
assume(epsilon, 'real')
assume(epsilon>0)

s = var ('s')
assume(s, 'real')
assume(s>0)

kappa = var ('kappa')
assume(kappa, 'real')
assume(kappa>0)

k = var ('k')
assume(k, 'real')

a = var('a')
assume(a, 'real')
assume(a>0)

x = var('x')
y = var('y')
z = var('z')
t = var('t')

assume(x, 'real')
assume(y, 'real')
assume(z, 'real')
assume(t, 'real')

c = var('c')
assume(c, 'real')
assume(c>0)

omega_t = var('omega')
assume(omega, 'real')
assume(omega > 0)

omega_t = var('omega_t')
assume(omega_t, 'real')

sigma = var('sigma')
assume(sigma, 'real')
assume(sigma > 0)

H_0 = var('H_0')
assume(H_0, 'real')

In [9]:
# а что если в данном расчете изменится сдвиг фаз между электрическим и магнитным полем?
phi = var("phi")
assume(phi, "real")

In [41]:
def get_m_phi(phase_shift_step = 0, logging = False):
    # глубина проникновения магнитного поля в проводник
    # Ландау Лифшиц. Электродинамика сплошных сред (59.4)
    delta = c / sqrt(2*pi*sigma*omega)
    if logging:
        display(Math(latex(delta)))

    # магнитное поле в проводнике
    # Ландау Лифшиц. Электродинамика сплошных сред (59.3)
    H_y(x) = H_0 * exp(-x/delta) * exp(I*(x/delta - omega * t))
    if logging:
        display(Math(latex(H_y(x))))
    re_H_y(x) = H_y(x).real_part()
    if logging:
        display(Math(latex(re_H_y(x))))

    # Ландау Лифшиц. Электродинамика сплошных сред (59.6)
    re_H_y(x) = H_0 * exp(-x/delta) * cos(x/delta - omega * t)
    if logging:
        display(Math(latex(re_H_y(x))))
    
    # делаем сдвиг фаз на угол (+ pi * phase_shift_step) + phi
    E_z_m(x) = H_y(x) * sqrt(omega/(8*pi*sigma)) * (1-I) * exp (I * phi) * exp (I * pi * phase_shift_step)
    if logging:
        show(E_z_m(x))

    # middle  arg(E/H) = - pi/4 + pi/2 + phi  = +0.25 + phi
    re_E_z_m(x) = H_0 * sqrt(omega/(4*pi*sigma)) * exp(-x/delta) * cos(x/delta - omega * t - pi/4 + pi*phase_shift_step + phi)
    if logging:
        display(Math(latex(re_E_z_m(x))))

    # ток проводимости вызванный электрическим полем
    # Ландау Лифшиц. Электродинамика сплошных сред (58.4)
    j_z_m(x) = sigma * re_E_z_m(x)
    if logging:
        show(j_z_m(x))

    # сила действующая на ток проводимости со стороны магнитного поля
    f_x_m(x) = 1 / c * j_z_m(x) * re_H_y(x)
    if logging:
        show(f_x_m(x))

    # делаем замену переменной для удобства интегрирования по периоду колебаний
    str_f_x_m = str(f_x_m(x))
    str_f_x_m = str_f_x_m.replace('omega*t', 'omega_t')
    f_x_s = []
    exec(preparse("f_x_s += [" + str_f_x_m + "]"))
    if logging:
        print(f_x_s[0](x = x))

    # усредняем по периоду колебаний
    F_x_m(x) = (1/(2*pi))*integrate(f_x_s[0](x=x), (omega_t, 0, 2*pi))
    if logging:
        show(F_x_m(x))
    
    # интегрируем по глубине
    int_F_x_m(x) = integrate(F_x_m(x), x)

    # силы действующей на проводник со стороны переменного магнитного поля
    F_m = limit(int_F_x_m(x).expand(), x=Infinity)
    if logging:
        show(F_m)
    F_m = -int_F_x_m(0).full_simplify().expand()
    if logging:
        show(F_m)
    
    # вычисляем силу действующую со стороны магнитного поля на ток Фуко в скин слое
    # вычисляем силу действующую со стороны магнитного поля на ток Фуко в скин слое
    # методом комплексного сопряжения
    f_conj_m = (1/2/c*sigma*E_z_m(x) * conjugate(H_y(x))).real()
    if logging:
        show(f_conj_m)
    F_conj_m = integrate(f_conj_m, x, 0, Infinity).full_simplify()
    if logging:
        show(F_conj_m)

    F_simp_m = (F_m.full_simplify()/(H_0^2/(16*pi))).expand()
    F_conj_simp_m = (F_conj_m.full_simplify()/(H_0^2/(16*pi))).expand()
    
    return F_conj_simp_m

In [59]:
#   phase_shift_step in [-2, -1.75, -1.5, -1.25, -1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2]:
#   phase_shift_step in [-2, -7/4,  -6/4, -5/4,  -1, -3/4, -1/2, -1/4, 0, 1/4, 1/2, 3/4, 1, 5/4, 6/4, 7/4, 2]:
for phase_shift_step in [-2,        -6/4,        -1,       -1/2,       0,      1/2,      1,      6/4,      2]:
    print(f"# arg(E/H) = - pi/4 + {phase_shift_step*pi + phi}:   \t : \t{get_m_phi(phase_shift_step = phase_shift_step)}")

# arg(E/H) = - pi/4 + -2*pi + phi:   	 : 	cos(phi) + sin(phi)
# arg(E/H) = - pi/4 + -3/2*pi + phi:   	 : 	cos(phi) - sin(phi)
# arg(E/H) = - pi/4 + -pi + phi:   	 : 	-cos(phi) - sin(phi)
# arg(E/H) = - pi/4 + -1/2*pi + phi:   	 : 	-cos(phi) + sin(phi)
# arg(E/H) = - pi/4 + phi:   	 : 	cos(phi) + sin(phi)
# arg(E/H) = - pi/4 + 1/2*pi + phi:   	 : 	cos(phi) - sin(phi)
# arg(E/H) = - pi/4 + pi + phi:   	 : 	-cos(phi) - sin(phi)
# arg(E/H) = - pi/4 + 3/2*pi + phi:   	 : 	-cos(phi) + sin(phi)
# arg(E/H) = - pi/4 + 2*pi + phi:   	 : 	cos(phi) + sin(phi)
